In [17]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from kss import split_sentences
import faiss
import numpy as np
from sklearn.neighbors import NearestNeighbors
import re
import requests


In [18]:
from hanspell import spell_checker
from soynlp.normalizer import *

from nltk.tokenize import word_tokenize

In [32]:
ETRI_API_KEY = "6e228975-a883-40c6-b197-6945c71cb977"

In [20]:
# 불용어 리스트
stopwords = ["이", "그", "저", "의", "을", "를", "은", "는", "에", "가", "와", "으로", "에서","책","추천"]

In [21]:
model_path = '/home/u4026/KoSentenceBERT_SKTBERT/output/training_sts'
model = SentenceTransformer(model_path)

using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_v1.zip
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Load Model


/home/u4026/.local/lib/python3.8/site-packages/sentence_transformers/models/Transformer.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.auto_model.load_state_dict(

In [22]:
nli_model_path = '/home/u4026/KoSentenceBERT_SKTBERT/output/training_nli'
nli_model = SentenceTransformer(nli_model_path)

using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_v1.zip
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Load Model


In [23]:
# CSV 파일 로드
csv_file = "books_summary.csv"
data = pd.read_csv(csv_file)

In [24]:
# 전처리된 텍스트 열 추출
corpus = data['내용요약'].tolist()
titles = data['첵제목'].tolist()

In [25]:
def get_synonyms2(word):
    url = "http://aiopen.etri.re.kr:8000/WiseNLU"
    headers = {
        "Authorization": ETRI_API_KEY,
        "Content-Type": "application/json"
    }
    data = {
        "argument": {
            "text": word,
            "analysis_code": "morp"
        }
    }

    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        result = response.json()
        # API 응답 데이터 구조에 따라 유의어 추출
        word_info = result.get("return_object", {}).get("sentence", [])
        synonyms = []
        for sentence in word_info:
            for morp in sentence.get("morp", []):
                if morp.get("type") == "NNG":  # 명사(NNG) 추출 예시
                    synonyms.append(morp.get("lemma"))
        return synonyms
    else:
        print(f"Error: {response.status_code}")
        return []

In [26]:
# 전처리 함수
def preprocess_text(text):
    # 1. 맞춤법 교정
    text = spell_checker.check(text).checked


    # 2. 특수문자 및 숫자 제거
    text = re.sub(r"[^가-힣0-9\s]", "", text)
    
    #반복되는 말 정규화
    text = repeat_normalize(text, num_repeats=2)

    # 3. 불용어 제거
    words = text.split()
    words = [word for word in words if word not in stopwords]

    # 4. 유의어 교체
    synonym_map = {}
    for word in words:
        synonyms = get_synonyms2(word)
        if synonyms:
            synonym_map[word] = synonyms[0]  # 첫 번째 유의어로 교체

    replaced_words = [synonym_map.get(word, word) for word in words]
    return " ".join(replaced_words)

In [27]:
# 내용요약을 문장별로 나누기
def split_corpus(corpus):
    split_sentences_corpus = []
    mapping = []  # 원본 데이터의 인덱스 매핑
    for idx, text in enumerate(corpus):
        sentences = split_sentences(text)
        split_sentences_corpus.extend(sentences)
        mapping.extend([idx] * len(sentences))
    return split_sentences_corpus, mapping

In [28]:
# 후보군 찾기 (KNN 활용)
def find_candidates_with_knn(query, split_corpus, mapping, titles, n_neighbors=3):
    # STS 임베딩 생성
    corpus_embeddings = model.encode(split_corpus)
    query_embedding = model.encode([query])

    # Nearest Neighbors 모델 초기화 및 학습
    knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    knn.fit(corpus_embeddings)

    # Query와 가장 가까운 이웃 찾기
    distances, indices = knn.kneighbors(query_embedding)

    candidates = []
    for dist, idx in zip(distances[0], indices[0]):
        original_idx = mapping[idx]
        candidates.append({
            "책제목": titles[original_idx],
            "내용요약": split_corpus[idx],
            "유사도": 1 - dist  # 코사인 거리 -> 유사도로 변환
        })

    # 유사도를 기준으로 정렬
    return sorted(candidates, key=lambda x: x["유사도"], reverse=True)


In [30]:
# 최종 유사도 평가 (STS 활용)
def evaluate_with_sts(query, candidates):
    query_embedding = model.encode([query], convert_to_tensor=True)
    results = []

    for candidate in candidates:
        corpus_embedding = model.encode([candidate['내용요약']], convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(query_embedding, corpus_embedding).item()

        results.append({
            "책제목": candidate['책제목'],
            "내용요약": candidate['내용요약'],
            "유사도": similarity
        })

    # 유사도를 기준으로 정렬
    return sorted(results, key=lambda x: x["유사도"], reverse=True)

In [33]:
pre_corpus = [preprocess_text(text) for text in corpus[:50]]
#test_corpus = "새 교육과정에 따른 평가기준의 개발이 필요해서, 한국교육과정평가원에서는 총론과 교과별 평가기준 등을 연구 개발 하였다.  핵심 개발 방향은 2015년 개정 교육과정이 학교현장에서 운영되도록 하는 것이다.한국과 독일 금형산업의 노동시장과 숙련형성 비교연구,,특성화고 전체 취업률은 약 45%, 마이스터고는 약 82%, 폴리텍은 약 86%, 2년제 전문대는 약 65%로 금형과 졸업자의 취업률은 학교별로 차이가 있다."
#pre_corpus= preprocess_text(test_corpus)

In [14]:
print(pre_corpus)

새 교육 따른 평가 기준 개발 필요 교육 총론 교과 평가 기준 등 연구 개발 핵심 개발 방향 2015년 개정 교육 학교 현장 운영 하는 것이다 한국과 독일 금형 노동 숙련 형성 비교 연구 특성 전체 취업 약 45 마이스터고는 약 82 폴리텍은 약 86 2년제 전문 약 65로 금형 졸업 취업 학교 차이 있다


In [34]:
# 사용자가 입력한 텍스트
query = input("텍스트를 입력하세요: ")
pre_query=preprocess_text(query)

텍스트를 입력하세요: 새 교육과정에 따른 평가내용이 담긴 책추천해줘


In [9]:
print(pre_query)

새 교육 따른 평가 내용 담긴 추천 줘


In [35]:


# Step 1: 내용요약을 문장별로 나누기
split_corpus_data, mapping = split_corpus(pre_corpus)

# Step 2: KNN으로 후보군 찾기
candidates = find_candidates_with_knn(pre_query, split_corpus_data, mapping, titles, n_neighbors=3)

# Step 3: STS로 최종 유사도 계산
final_results = evaluate_with_sts(query, candidates)

print(final_results)


[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



[{'책제목': '교육광장 2016 겨울호 (Vol. 62)', '내용요약': '새 교육 따른 평가 기준 개발 필요 교육 총론 교과 평가 기준 등 연구 개발 핵심 개발 방향 2015년 개정 교육 학교 현장 운영 하는 것이다', '유사도': 0.5710244178771973}, {'책제목': '미래 사회 교육 환경 변화에 따른 교과서 발전 방안 세미나', '내용요약': '교과 개발 대한 의견 교과 개발 주체 현재 다양 지식 형성 집단 지성 활용 현재 같은 방식 학생 교과 개발 참여 순 나타났다', '유사도': 0.48304980993270874}, {'책제목': '국민 삶의 질 측정 2016', '내용요약': '일부 국가 지표 작성 과정 하향 상향 보완 방식 사용 중 이러한 방식 통해 전문 중심 아닌 국민 참여 기반 국민 체감 높은 지표 작성 가능', '유사도': 0.39754611253738403}]
